In [1]:
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns


# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv

# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'missingno'

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
len(test)


418

In [6]:
len(train)

891

In [7]:
def clean(test):
    test = test.drop(["Name", "Ticket", "Cabin","Fare", "Age"], axis=1)
    return test

test= clean(test)

In [8]:
test.head(3)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked
0,892,3,male,0,0,Q
1,893,3,female,1,0,S
2,894,2,male,0,0,Q


In [9]:
train.isnull().sum().sort_values(ascending = False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [10]:
train.isnull().sum().sort_values(ascending = False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [11]:
train['Sex'].value_counts(dropna = False)


male      577
female    314
Name: Sex, dtype: int64

In [12]:
train[['Sex', 'Survived']].groupby('Sex', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [13]:
train[['Pclass', 'Survived']].groupby('Pclass', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [14]:
train[['SibSp', 'Survived']].groupby('SibSp', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [15]:
train[['Parch', 'Survived']].groupby('Parch', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [16]:
train[['Fare', 'Survived']].groupby('Fare', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

,Fare,Survived
247,512.3292,1.0
196,57.9792,1.0
89,13.8583,1.0
88,13.7917,1.0
86,13.4167,1.0
...,...,...
103,15.5500,0.0
180,47.1000,0.0
179,46.9000,0.0
178,42.4000,0.0


In [17]:
train[['Embarked', 'Survived']].groupby('Embarked', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


In [18]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
replace_Pclass = {1: 0.629630, 2: 0.472826, 3: 0.242363}
test['Pclass'] = test['Pclass'].replace(replace_Pclass)

In [20]:
test.head(3)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked
0,892,0.242363,male,0,0,Q
1,893,0.242363,female,1,0,S
2,894,0.472826,male,0,0,Q


In [21]:
replace_Sex = {'male': 0.188908, 'female': 0.742038}
test['Sex'] = test['Sex'].replace(replace_Sex)

In [22]:
replace_SibSp = {0: 0.345395, 1: 0.535885, 2: 0.464286, 3: 0.250000, 4: 0.166667, 5: 0.000000, 6: 0.000000}
test['SibSp'] = test['SibSp'].replace(replace_SibSp)

In [23]:
replace_Parch = {0: 0.343658, 1: 0.550847, 2: 0.500000, 3: 0.600000, 4: 0.000000, 5: 0.200000, 6: 0.000000}
test['Parch'] = test['Parch'].replace(replace_Parch)

In [24]:
replace_Embarked = {'C': 0.553571, 'Q': 0.389610, 'S': 0.336957}
test['Embarked'] = test['Embarked'].replace(replace_Embarked)

In [25]:
test.head(3)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked
0,892,0.242363,0.188908,0.345395,0.343658,0.389610
1,893,0.242363,0.742038,0.535885,0.343658,0.336957
2,894,0.472826,0.188908,0.345395,0.343658,0.389610


In [26]:

row_mean = test.iloc[0, 2:].mean()
test.head(3)
print(row_mean)

0.31689275


In [27]:
test['Mean'] = test.iloc[:, 1:].mean(axis=1)
test.head(3)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Mean
0,892,0.242363,0.188908,0.345395,0.343658,0.389610,0.301987
1,893,0.242363,0.742038,0.535885,0.343658,0.336957,0.440180
2,894,0.472826,0.188908,0.345395,0.343658,0.389610,0.348079


In [28]:
test['Survived?'] = (test['Mean']>=0.5).astype(int)
test.head(3)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Mean,Survived?
0,892,0.242363,0.188908,0.345395,0.343658,0.389610,0.301987,0
1,893,0.242363,0.742038,0.535885,0.343658,0.336957,0.440180,0
2,894,0.472826,0.188908,0.345395,0.343658,0.389610,0.348079,0


In [29]:
test.head(30)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Mean,Survived?
0,892,0.242363,0.188908,0.345395,0.343658,0.389610,0.301987,0
1,893,0.242363,0.742038,0.535885,0.343658,0.336957,0.440180,0
2,894,0.472826,0.188908,0.345395,0.343658,0.389610,0.348079,0
3,895,0.242363,0.188908,0.345395,0.343658,0.336957,0.291456,0
4,896,0.242363,0.742038,0.535885,0.550847,0.336957,0.481618,0
5,897,0.242363,0.188908,0.345395,0.343658,0.336957,0.291456,0
6,898,0.242363,0.742038,0.345395,0.343658,0.389610,0.412613,0
7,899,0.472826,0.188908,0.535885,0.550847,0.336957,0.417085,0
8,900,0.242363,0.742038,0.345395,0.343658,0.553571,0.445405,0
9,901,0.242363,0.188908,0.464286,0.343658,0.336957,0.315234,0


In [30]:
def clean(test):
    test_2 = test.drop(["Pclass", "Sex", "SibSp","Parch", "Embarked", "Mean"], axis=1)
    return test_2

test_2= clean(test)

In [31]:
test_2.head(3)

,PassengerId,Survived?
0,892,0
1,893,0
2,894,0


In [32]:
print(type(test_2))

<class 'pandas.core.frame.DataFrame'>


In [34]:
dsf = pd.DataFrame(test_2)
dsf.to_csv('Submission_data.csv', index=False)